In [28]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0,api_key=OPENAI_API_KEY)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [29]:
mail_generation_prompt=""" 
You are an expert executive outreach assistant. Your task is to generate a short, confident, and highly relevant outreach email from a candidate to a likely hiring manager. Your input is a structured alignment JSON that summarizes relevant insights from the candidate’s experience and the job’s requirements. You will NOT receive the full resume or job description — use only what is explicitly provided.

---

**EMAIL STRUCTURE TO FOLLOW:**

1. **Introduction**  
Hi, I came across this [role_title] position at [company_name]; while doing my research, I believe you're likely the hiring manager. Seeing a strong alignment, I thought I'd reach out to you directly.

2. **High-Level Alignment (Core Fit)**  
- Start with something like I see you are looking for... these kind of start 
- Clearly restate what the company is looking for using `primary_objective_alignment.job_specific_requirements`  
- Match it with `primary_objective_alignment.candidate_alignment and candidate strengths`, but use your own words for a smooth, confident flow  
- Use reference of one or more companies if helpful to convey scale or strategic fit.
- Use a polite tone.
- Create a completeness

🆕 Support Reasoning (Modified):
- Weave together 2–3 of the strongest experiences from high_level_alignment and candidate_strengths, ideally from multiple companies.
- Use natural transitions (“At [Company], I...”, “I’ve also...”) to make it a cohesive narrative — not bullet points. Keep it outcome-focused, technical when relevant, and fluid.

4. **Environment Fit **  (Optional)
- Briefly use `environment_fit.candidate_experience` to show the candidate is used to operating in similarly scaled or complex environments  
- Make this paragraph no more than 1–2 sentences  
- Skip this section if it adds no new insight

5. **Call to Action**  
If you open my resume, you'll see a striking resonance in what you're looking for. I have already applied to this position and would love to have a quick conversation if you're still hiring!

6. **Signature**  
Best,  
[candidate_name]

---

**STYLE & TONE REQUIREMENTS:**

- Confident and peer-to-peer tone — not overly formal or apologetic  
- Avoid buzzwords, clichés, and vague generalizations  
- Never refer to a “resume” or “job description” being read — treat everything as known insight  
- Always ground examples in **what’s in the JSON** — no speculation  
- Weave in **multiple companies if applicable**, and **prioritize impact**

---

**INPUT FORMAT (JSON):**

You will always receive a single JSON object with the following fields:

```json

  "role_title": "string",
  "company_name": "string",
  "candidate_name": "string",
  "primary_objective_alignment": 
    "job_specific_requirements": "string",
    "candidate_alignment": "string"
  ,
  "high_level_alignment": [
    
      "alignment_type": "string",
      "candidate_experience": "string"
    
  ],
  "environment_fit": 
    "company_size": "string",
    "industry": "string",
    "customer_type": "string",
    "candidate_experience": "string"
  ,
  "candidate_strengths": [
    
      "strength": "string",
      "candidate_experience": "string"
    
  ],
  "gaps_identified": 
    "gap": "string"
  

RULES:

Do NOT invent details beyond the input

Do NOT overfocus on a single company unless justified by the content

Do NOT include the gaps_identified field — ignore it entirely

Do NOT reference the resume or job description directly — only refer to alignment
 
"""

In [30]:
from langgraph.prebuilt import create_react_agent
prompt = ChatPromptTemplate.from_messages([
     ("system", mail_generation_prompt),
     ("placeholder", "{messages}"),
     ("user", "Always be accurate."),
 ])
tools=[]
graph = create_react_agent(model, tools, prompt=prompt)


In [31]:
alignments=""" 
 {
  "primary_objective_alignment": {
    "job_specific_requirements": "The job requires developing and leading an enterprise-wide network security program, with a focus on advanced network security solutions, public cloud security (specifically AWS), and leadership in network security engineering.",
    "candidate_alignment": "Keith L. Oufnac has extensive experience in leading digital transformation and cybersecurity initiatives, as demonstrated in his role at Bollinger Shipyards where he directed network re-architecture and implemented key security measures, successfully passing a DIBCAC audit. His experience with Azure Gov Cloud and AWS aligns with the job's requirement for public cloud security expertise."
  },
  "high_level_alignment": [
    {
      "alignment_type": "Network Security Leadership",
      "candidate_experience": "At Bollinger Shipyards, Keith led the digital transformation and cybersecurity initiatives across 16 locations, supporting over 2,000 users, which aligns with the job's requirement for leadership in network security engineering."
    },
    {
      "alignment_type": "Cloud Security Expertise",
      "candidate_experience": "Keith has hands-on experience with AWS and Azure Gov Cloud, having led cloud migration initiatives at Marsh & McLennan Agency, ensuring business continuity and operational efficiency, which matches the job's focus on public cloud security."
    },
    {
      "alignment_type": "Incident and Change Management",
      "candidate_experience": "Keith's role at Marsh & McLennan Agency involved overseeing IT due diligence and integration for mergers and acquisitions, which required managing high availability SLAs and incident response, aligning with the job's need for experience in incident and change management in network security operations."
    }
  ],
  "environment_fit": {
    "company_size": "Large Financial Services Firm",
    "industry": "Financial Services",
    "customer_type": "B2B",
    "candidate_experience": "Keith has experience in large-scale environments, such as Bollinger Shipyards and Marsh & McLennan Agency, where he led IT and security initiatives, aligning with LPL Financial's need for a leader in a large financial services firm."
  },
  "candidate_strengths": [
    {
      "strength": "Digital Transformation Leadership",
      "candidate_experience": "Successfully led digital transformation initiatives at Bollinger Shipyards, including infrastructure modernization and cybersecurity, which aligns with the job's emphasis on leading an enterprise-wide network security program."
    },
    {
      "strength": "Cloud and Network Security Expertise",
      "candidate_experience": "Keith's experience with AWS, Azure Gov Cloud, and advanced network security solutions like Cisco Meraki and Fortinet directly supports the job's requirements for public cloud security and advanced network security solutions."
    },
    {
      "strength": "Operational Leadership",
      "candidate_experience": "Managed IT due diligence and integration for mergers and acquisitions at Marsh & McLennan Agency, demonstrating his ability to lead complex operations and manage high availability SLAs, which is crucial for the role's operational scale."
    }
  ]
}

"""

In [32]:
user_input=f"key allignments between candidate's resume and job description are: \n\n {alignments}" 

In [33]:
inputs = {"messages": [("user", user_input)]}
for s in graph.stream(inputs, stream_mode="values"):
     message = s["messages"][-1]
     if isinstance(message, tuple):
         print(message)
     else:
         message.pretty_print()

================================ Human Message =================================

key allignments between candidate's resume and job description are: 

  
 {
  "primary_objective_alignment": {
    "job_specific_requirements": "The job requires developing and leading an enterprise-wide network security program, with a focus on advanced network security solutions, public cloud security (specifically AWS), and leadership in network security engineering.",
    "candidate_alignment": "Keith L. Oufnac has extensive experience in leading digital transformation and cybersecurity initiatives, as demonstrated in his role at Bollinger Shipyards where he directed network re-architecture and implemented key security measures, successfully passing a DIBCAC audit. His experience with Azure Gov Cloud and AWS aligns with the job's requirement for public cloud security expertise."
  },
  "high_level_alignment": [
    {
      "alignment_type": "Network Security Leadership",
      "candidate_experience": 

first message :

Hi, I came across this VP of IT position at Panda Restaurant Group; while doing my research, I believe you're likely the hiring manager. Seeing a strong alignment, I thought I'd reach out to you directly.

I see you're looking to provide strategic leadership to support company growth, ensure stability and security of infrastructure, and manage infrastructure spending effectively. Having led digital transformation and infrastructure modernization at Bollinger Shipyards, where I reduced infrastructure costs by 20% and improved network performance by 30%, I believe I'd be a solid fit.

At Bollinger Shipyards, I successfully migrated our systems to Azure Gov Cloud, which directly reduced infrastructure costs by 20%. Additionally, as VP of IT, I led digital transformation initiatives across 16 locations, supporting over 2,000 users, which aligns with your need for leadership in IS management.

While my experience is primarily in large-scale environments like Bollinger Shipyards, the complexity and scale of operations there align well with the operational scale of Panda Restaurant Group.

If you open my resume, you'll see a striking resonance in what you're looking for. I have already applied to this position and would love to have a quick conversation if you're still hiring!

Best,  
Keith L. Oufnac

Second message :



Hi, I came across this strategic leadership position at Panda Restaurant Group; while doing my research, I believe you're likely the hiring manager. Seeing a strong alignment, I thought I'd reach out to you directly.

The role requires strategic leadership to support company growth, ensuring stability and security of infrastructure, and effective cost management. At Bollinger Shipyards, I led digital transformation and infrastructure modernization, reducing costs by 20% and improving network performance by 30%, which aligns well with your needs.

In my role as VP of IT at Bollinger Shipyards, I spearheaded digital transformation initiatives across 16 locations, supporting over 2,000 users. This experience demonstrates my capability in leadership and infrastructure management. Additionally, my expertise in cloud technologies, including successful migrations to Azure Gov Cloud and AWS, is directly relevant to overseeing cloud and infrastructure service providers.

While my experience is primarily in large-scale environments like Bollinger Shipyards, the complexity and scale of operations align with those at Panda Restaurant Group.

If you open my resume, you'll see a striking resonance in what you're looking for. I have already applied to this position and would love to have a quick conversation if you're still hiring!

Best,  
Keith L. Oufnac


Hi, I came across this IT Infrastructure and Operations Leader position at BusPatrol; while doing my research, I believe you're likely the hiring manager. Seeing a strong alignment, I thought I'd reach out to you directly.

BusPatrol is looking for someone to lead global IT infrastructure and operations through digital transformation, with a focus on cloud infrastructure, DevOps, and security. At Bollinger Shipyards, I spearheaded digital transformation initiatives, successfully migrating our systems to Azure Gov Cloud and enhancing network performance by 30%. This experience aligns well with your need for cloud infrastructure expertise. I’ve also led cybersecurity initiatives, including passing a DIBCAC audit, which underscores my capability in IT security—a critical aspect of your role.

Additionally, my experience at Marsh & McLennan Agency, where I utilized AWS for cloud migration initiatives, further solidifies my cloud computing expertise. I have managed IT operations across multiple locations and led IT integration for major acquisitions, demonstrating my ability to lead cross-functional teams and manage large-scale IT projects, which is crucial for the global leadership requirement at BusPatrol.

I have experience in large-scale IT environments, leading digital transformation and IT operations, which aligns with BusPatrol's need for leadership in smart transportation technology.

If you open my resume, you'll see a striking resonance in what you're looking for. I have already applied to this position and would love to have a quick conversation if you're still hiring!

Best,  
Keith L. Oufnac


Hi, I came across this Technology Infrastructure Strategy position at J.S. Held LLC; while doing my research, I believe you're likely the hiring manager. Seeing a strong alignment, I thought I'd reach out to you directly.

The role requires leading the technology infrastructure strategy with a focus on Azure and Microsoft infrastructure, ensuring security, availability, integrity, and confidentiality. At Bollinger Shipyards, I spearheaded a digital transformation and infrastructure modernization, including a migration to Azure Gov Cloud, which reduced infrastructure costs by 20% and enhanced system uptime by 15%. This experience aligns well with your need for Azure expertise and strategic infrastructure leadership. I’ve also successfully passed the DIBCAC audit with no significant issues, demonstrating my capability in ensuring security and compliance, which is crucial for your role.

Additionally, I have led infrastructure modernization across 16 locations, supporting over 2,000 users, and have extensive experience with Microsoft 365 GCCH. My background in large-scale environments and IT operations in acquisitive organizations aligns with J.S. Held LLC's global consulting operations.

If you open my resume, you'll see a striking resonance in what you're looking for. I have already applied to this position and would love to have a quick conversation if you're still hiring!

Best,  
Keith L. Oufnac

In [ ]:
sixth message

Hi, I came across this Network Security Program Lead position at LPL Financial; while doing my research, I believe you're likely the hiring manager. Seeing a strong alignment, I thought I'd reach out to you directly.

I see you are looking for someone to develop and lead an enterprise-wide network security program, focusing on advanced network security solutions and public cloud security, particularly AWS. My experience aligns well with these requirements. At Bollinger Shipyards, I directed network re-architecture and implemented key security measures, successfully passing a DIBCAC audit. This role involved leading digital transformation and cybersecurity initiatives across 16 locations, supporting over 2,000 users, which speaks to my leadership in network security engineering. I’ve also led cloud migration initiatives at Marsh & McLennan Agency, ensuring business continuity and operational efficiency with AWS and Azure Gov Cloud, directly matching your focus on public cloud security.

Additionally, my experience in large-scale environments, such as Bollinger Shipyards and Marsh & McLennan Agency, where I led IT and security initiatives, aligns with LPL Financial's need for a leader in a large financial services firm.

If you open my resume, you'll see a striking resonance in what you're looking for. I have already applied to this position and would love to have a quick conversation if you're still hiring!

Best,  
Keith L. Oufnac